In [1]:
# Fake News Detection Using Python using ensemble techniques
import pandas as pd
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Loading the datasets
data_true = pd.read_csv("True.csv")
data_fake = pd.read_csv("Fake.csv")
data_true.sample(5)

C:\Users\Shariph\AppData\Local\Temp\ipykernel_4744\885832482.py:3: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  data_fake = pd.read_csv("Fake.csv")


,title,text,subject,date
11320,Hundreds sacked from Uzbek FinMin in 'rat' purge,TASHKENT (Reuters) - Uzbekistan s Finance Mini...,worldnews,"December 27, 2017"
19151,Cambodia's opposition puts out banners calling...,PHNOM PENH (Reuters) - Cambodia s main opposit...,worldnews,"September 25, 2017"
9349,"Illinois governor blasts House budget, Chicago...","SPRINGFIELD, Ill. (Reuters) - Just days before...",politicsNews,"May 29, 2016"
9079,Clinton's lead over Trump slips after Florida ...,NEW YORK (Reuters) - Donald Trump chipped away...,politicsNews,"June 17, 2016"
4228,Trump advisers to meet Tuesday to discuss Pari...,WASHINGTON (Reuters) - Advisers to President D...,politicsNews,"April 17, 2017"


In [3]:
data_fake.sample(5)

,title,text,subject,date,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128
13566,BREAKING DALLAS: DPD Chief Confirms 10 Officer...,UPDATE 12:09: 1 Suspect in custody. 1 suspect ...,politics,"Jul 7, 2016",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11259,REPORT: FBI DIRECTOR COMEY Blocked By Obama Ad...,Comey would likely have pitched the op-ed to T...,politics,"Mar 30, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5755,Louie Gohmert Gets Completely Unhinged And Sc...,"Whenever big news happens in Washington, the n...",News,"June 23, 2016",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2998,Atlanta Mayor And Newspaper DESTROY Trump For...,Donald Trump called Atlanta a crime infested ...,News,"January 15, 2017",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17391,"BREAKING: CHARITY FAILED TO REVEAL 1,100 DONOR...",More and more dirt on these two grifters who r...,Government News,"Apr 29, 2015",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Dropping the Unnamed Columns
unnamed_columns = [col for col in data_fake.columns if col.startswith('Unnamed')]
data_fake.drop(columns = unnamed_columns, inplace = True)
data_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
22893,Washington’s Criminal Activities Are Only Gett...,21st Century Wire says It was a major scandal ...,Middle-east,"May 9, 2017"
22894,Will ‘Trumponomics’ Bankrupt America?,21st Century Wire says F. William Engdahl is a...,Middle-east,"May 4, 2017"
22895,The Existential Question Of Whom To Trust,21st Century Wire says Investigative reporter ...,Middle-east,"May 1, 2017"
22896,Boiler Room #106 – Did Israel Attack Damascus?...,Tune in to the Alternate Current Radio Network...,Middle-east,"April 28, 2017"


In [5]:
# Adding the classes to classify the data is fake or true
data_fake["Class"] = 1
data_true["Class"] = 0

In [6]:
# Now, Let's Concatinate the datasets
data = pd.concat([data_true, data_fake], axis = 0)
data

,title,text,subject,date,Class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
22893,Washington’s Criminal Activities Are Only Gett...,21st Century Wire says It was a major scandal ...,Middle-east,"May 9, 2017",1
22894,Will ‘Trumponomics’ Bankrupt America?,21st Century Wire says F. William Engdahl is a...,Middle-east,"May 4, 2017",1
22895,The Existential Question Of Whom To Trust,21st Century Wire says Investigative reporter ...,Middle-east,"May 1, 2017",1
22896,Boiler Room #106 – Did Israel Attack Damascus?...,Tune in to the Alternate Current Radio Network...,Middle-east,"April 28, 2017",1


In [7]:
# Now let's Do some Modeling
# First Let's Train the datsets
data.dropna(subset=['text'], inplace=True)
X = data['text']
y = data['Class']
data

,title,text,subject,date,Class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
22893,Washington’s Criminal Activities Are Only Gett...,21st Century Wire says It was a major scandal ...,Middle-east,"May 9, 2017",1
22894,Will ‘Trumponomics’ Bankrupt America?,21st Century Wire says F. William Engdahl is a...,Middle-east,"May 4, 2017",1
22895,The Existential Question Of Whom To Trust,21st Century Wire says Investigative reporter ...,Middle-east,"May 1, 2017",1
22896,Boiler Room #106 – Did Israel Attack Damascus?...,Tune in to the Alternate Current Radio Network...,Middle-east,"April 28, 2017",1


In [8]:
# Transforming
cv = CountVectorizer()
X = cv.fit_transform(X)

In [9]:
# Training the datasets
xtrain,xtest,ytrain,ytest = train_test_split(X,y)

In [10]:
# 1. Logistic Regression
logr = LogisticRegression()
logr.fit(xtrain,ytrain)
logr.score(xtest,ytest)

C:\Users\Shariph\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9953055881556379

In [11]:
# 2. Decision Tree Classifier
dtree = DecisionTreeClassifier()
dtree.fit(xtrain,ytrain)
dtree.score(xtest,ytest)

0.9962986368150221

In [12]:
# 3. MultinomialNB | Naive_Bayes
model = MultinomialNB()
model.fit(xtrain,ytrain)
model.score(xtest,ytest)

0.9518822785952875

In [13]:
# Now let's Detect the Fake news
user = input("Enter: ")
user_input = cv.transform([user]).toarray()
# Output

# Let's Define the functions for this tasks
def output():
    print(logr.predict(user_input))
    print(dtree.predict(user_input))
    print(model.predict(user_input))
output()

Enter:  Federal health officials told the AP they have not received any reports of Ebola cases at the Nevada event. A screenshot of a supposed post from the Centers for Disease Control and Prevention confirming such cases was fabricated. And there is no record of a national emergency being declared. The claims emerged after summer storm left muddy roads flooded, stranding tens of thousands of partygoers; event organizers let traffic flow out of the main road Monday afternoon. “So it was announced earlier that Burning Man was declared a national emergency because it was flooded, and so they sent in FEMA,” a woman claims in a TikTok video shared on Instagram, suggesting the development was suspicious. The AP found no record, including on federal websites and in White House announcements, of a national emergency declaration and FEMA confirmed that it was not involved in the situation. “No FEMA personnel or assets have been deployed to the Burning Man festival and there are no requests fro

[1]
[1]
[1]
